In [14]:
import sys
sys.path.insert(0,'../src/')
from curve import *
from utils import *
from multiply_connected_pipe import *
import pickle

from scipy.sparse.linalg import lgmres

from time import time

with open('global_pipe_with_geometry.pickle','rb') as f:
    pipe = pickle.load(f)

required_tol = 1e-10
pipe.build_A(fmm=True)
print("number of points: ", len(pipe.t))

number of points:  497042


In [15]:
b = np.concatenate([pipe.boundary_value(0).real, pipe.boundary_value(0).imag],dtype=np.float64)
A = pipe.A

In [16]:
class Callback:
    
    def __init__(self):
        self.iter = 0
        self.residuals = []
        self.t = time()
        
    def __call__(self, xk):
        self.iter += 1
        if self.iter % 20 == 0:
            residual = np.linalg.norm(A(xk) - b)
            self.residuals.append(residual)
            print(f"iter {self.iter}", 
                  f"resdiual {residual}", 
                  f"time {int((time()-self.t)/60)} min", 
                  sep='\t')

In [17]:
omega_sep, _ = lgmres(A, b, 
                    atol=0, tol=required_tol,
                    inner_m=500, outer_k=10 , maxiter=100000, 
                    callback=Callback())

KeyboardInterrupt: 